data 

train_data_ix_{vocab_size}_{n}gram.pickle val_data_ix_{vocab_size}_{n}gram.pickle test_data_ix_{vocab_size}_{n}gram.pickle

In [15]:
train_label[:10]

[array([0.]),
 array([0.]),
 array([0.]),
 array([0.]),
 array([0.]),
 array([0.]),
 array([0.]),
 array([0.]),
 array([0.]),
 array([0.])]

In [13]:
train_label = list(np.zeros((25000,1))) + list(np.ones((25000,1)))

In [36]:
import glob
import numpy as np
from pprint import pprint
import random

def shuffle_data(data, targets):    
    data_zip = list(zip(data, targets))
    random.shuffle(data_zip)
    data_shuffle, targets_shuffle = list(zip(*train_zip))
    return data_shuffle, targets_shuffle

# load data
data_path = 'data'

def load_data(kind):
    data = []
    
    for label in ('neg', 'pos'):
        files = glob.glob(data_path + f"/{kind}/{label}/*")
        assert(len(files) == 12500)
        for text_file in files:
            with open(text_file, 'r') as fin:
                data.append(fin.readline().strip())
                
    assert(len(data) == 25000)
    n_pos = 12500
    targets = list(np.zeros((n_pos))) + list(np.ones((n_pos)))
    
    return shuffle_data(data, targets)

train_data_pre, train_targets_pre = load_data('train')
test_data, test_targets = load_data('test')

In [37]:
# Split train data into actual train and validation sets
print ("Train dataset size is {}".format(len(train_data_pre)))
print ("Test dataset size is {}".format(len(test_data)))

train_split = 20000

val_data = train_data_pre[train_split:]
val_targets = train_targets_pre[train_split:]

train_data = train_data_pre[:train_split]
train_targets = train_targets_pre[:train_split]

print("After val split")
print ("Train dataset size is {}".format(len(train_data)))
print ("Val dataset size is {}".format(len(val_data)))
print ("Test dataset size is {}".format(len(test_data)))

Train dataset size is 25000
Test dataset size is 25000
After val split
Train dataset size is 20000
Val dataset size is 5000
Test dataset size is 25000


In [38]:
# Random sample from train dataset
ix = random.randint(0, len(train_data) - 1)
print(f"Target: {train_targets[ix]}\n")
print (train_data[ix])

Target: 1.0

I decided to watch this because of the recommendations from this site. I would have to say it was worth the effort. However, you should take heed that this film will go on for 210 minutes. If you don't have the staying power, get it on tape and watch it over a couple of nights.<br /><br />Now to the film, what I say will contain "spoilers" and if you don't mind, here goes: <br /><br />Alexandre is a promiscuous bum, a womanizer and a gigolo. He lives with an older woman called Marie. Marie owns a retail shop and she provides for Alex. Alex spends his days at cafés and restaurants. The story reveals that Alex had previously impregnated Gilberte whom he used to live with. Gilberte dumped him for a less attractive man that she did not love because Alex had abused and battered her. At this point, Alex was willing to get a job and and help raise their child before he found out Gilberte had aborted it and planned to marry someone else. <br /><br />By chance, Alexandre meets a nu

Before training the classifier, first we are going to tokenize the dataset using spacy.io

Run (shown in the cell below):

* ```pip install spacy```
* ```python -m spacy download en_core_web_sm```

In [4]:
!pip install spacy
!python -m spacy download en_core_web_sm

smart-open 1.5.6 requires boto3, which is not installed.
You are using pip version 10.0.0, however version 18.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 10.0.0, however version 18.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.

    Linking successful
    /Users/diogomesquita/anaconda/envs/py3.6/lib/python3.6/site-packages/en_core_web_sm
    -->
    /Users/diogomesquita/anaconda/envs/py3.6/lib/python3.6/site-packages/spacy/data/en_core_web_sm

    You can now load the model via spacy.load('en_core_web_sm')



In [17]:
# Let's write the tokenization function 

import spacy
import string

# Load English tokenizer, tagger, parser, NER and word vectors
tokenizer = spacy.load('en_core_web_sm')
punctuations = string.punctuation

# lowercase and remove punctuation
def tokenize(sent):
  tokens = tokenizer(sent)
  return [token.text.lower() for token in tokens if (token.text not in punctuations)]

# Example
tokens = tokenize(u'Apple is, looking ! at buying U.K. startup for $1 billion')
print (tokens)

['apple', 'is', 'looking', 'at', 'buying', 'u.k.', 'startup', 'for', '1', 'billion']


In [6]:
# This is the code cell that tokenizes train/val/test datasets
# However it takes about 15-20 minutes to run it
# For convinience we have provided the preprocessed datasets
# Please see the next code cell
import pickle as pkl

def tokenize_dataset(dataset):
    token_dataset = []
    # we are keeping track of all tokens in dataset 
    # in order to create vocabulary later
    all_tokens = []
    
    for sample in dataset:
        tokens = tokenize(sample)
        token_dataset.append(tokens)
        all_tokens += tokens

    return token_dataset, all_tokens

# val set tokens
#print ("Tokenizing val data")
#val_data_tokens, _ = tokenize_dataset(val_data)
#pkl.dump(val_data_tokens, open("val_data_tokens.p", "wb"))

# test set tokens
#print ("Tokenizing test data")
#test_data_tokens, _ = tokenize_dataset(test_data)
#pkl.dump(test_data_tokens, open("test_data_tokens.p", "wb"))

# train set tokens
#print ("Tokenizing train data")
#train_data_tokens, all_train_tokens = tokenize_dataset(train_data)
#pkl.dump(train_data_tokens, open("train_data_tokens.p", "wb"))
#pkl.dump(all_train_tokens, open("all_train_tokens.p", "wb"))

In [7]:
# Alternatively try running the following multi-threaded version of tokenization
# Credit to Ilya Kulikov

def lower_case_remove_punc(parsed):
    return [token.text.lower() for token in parsed if (token.text not in punctuations)]

def tokenize_dataset(dataset):
    token_dataset = []
    # we are keeping track of all tokens in dataset
    # in order to create vocabulary later
    all_tokens = []

    for sample in tqdm_notebook(tokenizer.pipe(dataset, disable=['parser', 'tagger', 'ner'], batch_size=512, n_threads=1)):
        tokens = lower_case_remove_punc(sample)
        token_dataset.append(tokens)
        all_tokens += tokens

    return token_dataset, all_tokens

# val set tokens
#print ("Tokenizing val data")
#val_data_tokens, _ = tokenize_dataset(val_data)
#pkl.dump(val_data_tokens, open("val_data_tokens.p", "wb"))

# test set tokens
#print ("Tokenizing test data")
#test_data_tokens, _ = tokenize_dataset(test_data)
#pkl.dump(test_data_tokens, open("test_data_tokens.p", "wb"))

# train set tokens
#print ("Tokenizing train data")
#train_data_tokens, all_train_tokens = tokenize_dataset(train_data)
#pkl.dump(train_data_tokens, open("train_data_tokens.p", "wb"))
#pkl.dump(all_train_tokens, open("all_train_tokens.p", "wb"))

In [8]:
# First, download datasets from here
# Use your NYU account
#https://drive.google.com/open?id=1eR2LFI5MGliHlaL1S2nsX4ouIO1k_ip2
#https://drive.google.com/open?id=133QCWbiz_Xc7Qm4r6t-fJP1K669xjNlM
#https://drive.google.com/open?id=1SuUIUpJ1iznU707ktkpnEGSwt_XIqOYp
#https://drive.google.com/open?id=1UQsrZ2LVfcxdxxa47344fMs_qvya72KR
import pickle as pkl
# Then, load preprocessed train, val and test datasets
train_data_tokens = pkl.load(open("train_data_tokens.p", "rb"))
all_train_tokens = pkl.load(open("all_train_tokens.p", "rb"))

val_data_tokens = pkl.load(open("val_data_tokens.p", "rb"))
test_data_tokens = pkl.load(open("test_data_tokens.p", "rb"))

# double checking
print ("Train dataset size is {}".format(len(train_data_tokens)))
print ("Val dataset size is {}".format(len(val_data_tokens)))
print ("Test dataset size is {}".format(len(test_data_tokens)))

print ("Total number of tokens in train dataset is {}".format(len(all_train_tokens)))

Train dataset size is 10000
Val dataset size is 1314
Test dataset size is 7532
Total number of tokens in train dataset is 3433739


Now, we are going to create the vocabulary of most common 10,000 tokens in the training set.

In [9]:
from collections import Counter

max_vocab_size = 10000
# save index 0 for unk and 1 for pad
PAD_IDX = 0
UNK_IDX = 1

def build_vocab(all_tokens):
    # Returns:
    # id2token: list of tokens, where id2token[i] returns token that corresponds to token i
    # token2id: dictionary where keys represent tokens and corresponding values represent indices
    token_counter = Counter(all_tokens)
    vocab, count = zip(*token_counter.most_common(max_vocab_size))
    id2token = list(vocab)
    token2id = dict(zip(vocab, range(2,2+len(vocab)))) 
    id2token = ['<pad>', '<unk>'] + id2token
    token2id['<pad>'] = PAD_IDX 
    token2id['<unk>'] = UNK_IDX
    return token2id, id2token

token2id, id2token = build_vocab(all_train_tokens)

In [10]:
# Lets check the dictionary by loading random token from it

random_token_id = random.randint(0, len(id2token)-1)
random_token = id2token[random_token_id]

print ("Token id {} ; token {}".format(random_token_id, id2token[random_token_id]))
print ("Token {}; token id {}".format(random_token, token2id[random_token]))

Token id 2135 ; token recommend
Token recommend; token id 2135


In [11]:
# convert token to id in the dataset
def token2index_dataset(tokens_data):
    indices_data = []
    for tokens in tokens_data:
        index_list = [token2id[token] if token in token2id else UNK_IDX for token in tokens]
        indices_data.append(index_list)
    return indices_data

train_data_indices = token2index_dataset(train_data_tokens)
val_data_indices = token2index_dataset(val_data_tokens)
test_data_indices = token2index_dataset(test_data_tokens)

# double checking
print ("Train dataset size is {}".format(len(train_data_indices)))
print ("Val dataset size is {}".format(len(val_data_indices)))
print ("Test dataset size is {}".format(len(test_data_indices)))

Train dataset size is 10000
Val dataset size is 1314
Test dataset size is 7532


Now we are going to create PyTorch DataLoader 

In [12]:
MAX_SENTENCE_LENGTH = 200

import numpy as np
import torch
from torch.utils.data import Dataset

class NewsGroupDataset(Dataset):
    """
    Class that represents a train/validation/test dataset that's readable for PyTorch
    Note that this class inherits torch.utils.data.Dataset
    """
    
    def __init__(self, data_list, target_list):
        """
        @param data_list: list of newsgroup tokens 
        @param target_list: list of newsgroup targets 

        """
        self.data_list = data_list
        self.target_list = target_list
        assert (len(self.data_list) == len(self.target_list))

    def __len__(self):
        return len(self.data_list)
        
    def __getitem__(self, key):
        """
        Triggered when you call dataset[i]
        """
        
        token_idx = self.data_list[key][:MAX_SENTENCE_LENGTH]
        label = self.target_list[key]
        return [token_idx, len(token_idx), label]

def newsgroup_collate_func(batch):
    """
    Customized function for DataLoader that dynamically pads the batch so that all 
    data have the same length
    """
    data_list = []
    label_list = []
    length_list = []
    #print("collate batch: ", batch[0][0])
    #batch[0][0] = batch[0][0][:MAX_SENTENCE_LENGTH]
    for datum in batch:
        label_list.append(datum[2])
        length_list.append(datum[1])
    # padding
    for datum in batch:
        padded_vec = np.pad(np.array(datum[0]), 
                                pad_width=((0,MAX_SENTENCE_LENGTH-datum[1])), 
                                mode="constant", constant_values=0)
        data_list.append(padded_vec)
    return [torch.from_numpy(np.array(data_list)), torch.LongTensor(length_list), torch.LongTensor(label_list)]

# create pytorch dataloader
#train_loader = NewsGroupDataset(train_data_indices, train_targets)
#val_loader = NewsGroupDataset(val_data_indices, val_targets)
#test_loader = NewsGroupDataset(test_data_indices, test_targets)

BATCH_SIZE = 32
train_dataset = NewsGroupDataset(train_data_indices, train_targets)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=BATCH_SIZE,
                                           collate_fn=newsgroup_collate_func,
                                           shuffle=True)

val_dataset = NewsGroupDataset(val_data_indices, val_targets)
val_loader = torch.utils.data.DataLoader(dataset=val_dataset, 
                                           batch_size=BATCH_SIZE,
                                           collate_fn=newsgroup_collate_func,
                                           shuffle=True)

test_dataset = NewsGroupDataset(test_data_indices, test_targets)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                           batch_size=BATCH_SIZE,
                                           collate_fn=newsgroup_collate_func,
                                           shuffle=False)

#for i, (data, lengths, labels) in enumerate(train_loader):
#    print (data)
#    print (labels)
#    break

Here we will define Bag-of-Words model in PyTorch

In [13]:
# First import torch related libraries
import torch
import torch.nn as nn
import torch.nn.functional as F

class BagOfWords(nn.Module):
    """
    BagOfWords classification model
    """
    def __init__(self, vocab_size, emb_dim):
        """
        @param vocab_size: size of the vocabulary. 
        @param emb_dim: size of the word embedding
        """
        super(BagOfWords, self).__init__()
        # pay attention to padding_idx 
        self.embed = nn.Embedding(vocab_size, emb_dim, padding_idx=0)
        self.linear = nn.Linear(emb_dim,20)
    
    def forward(self, data, length):
        """
        
        @param data: matrix of size (batch_size, max_sentence_length). Each row in data represents a 
            review that is represented using n-gram index. Note that they are padded to have same length.
        @param length: an int tensor of size (batch_size), which represents the non-trivial (excludes padding)
            length of each sentences in the data.
        """
        out = self.embed(data)
        out = torch.sum(out, dim=1)
        out /= length.view(length.size()[0],1).expand_as(out).float()
     
        # return logits
        out = self.linear(out.float())
        return out

emb_dim = 100
model = BagOfWords(len(id2token), emb_dim)

In [14]:
learning_rate = 0.01
num_epochs = 10 # number epoch to train

# Criterion and Optimizer
criterion = torch.nn.CrossEntropyLoss()  
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Function for testing the model
def test_model(loader, model):
    """
    Help function that tests the model's performance on a dataset
    @param: loader - data loader for the dataset to test against
    """
    correct = 0
    total = 0
    model.eval()
    for data, lengths, labels in loader:
        data_batch, length_batch, label_batch = data, lengths, labels
        outputs = F.softmax(model(data_batch, length_batch), dim=1)
        predicted = outputs.max(1, keepdim=True)[1]
        
        total += labels.size(0)
        correct += predicted.eq(labels.view_as(predicted)).sum().item()
    return (100 * correct / total)

for epoch in range(num_epochs):
    for i, (data, lengths, labels) in enumerate(train_loader):
        model.train()
        data_batch, length_batch, label_batch = data, lengths, labels
        optimizer.zero_grad()
        outputs = model(data_batch, length_batch)
        loss = criterion(outputs, label_batch)
        loss.backward()
        optimizer.step()
        # validate every 100 iterations
        if i > 0 and i % 100 == 0:
            # validate
            val_acc = test_model(val_loader, model)
            print('Epoch: [{}/{}], Step: [{}/{}], Validation Acc: {}'.format( 
                       epoch+1, num_epochs, i+1, len(train_loader), val_acc))


Epoch: [1/10], Step: [101/313], Validation Acc: 44.67275494672755
Epoch: [1/10], Step: [201/313], Validation Acc: 69.17808219178082
Epoch: [1/10], Step: [301/313], Validation Acc: 79.60426179604262
Epoch: [2/10], Step: [101/313], Validation Acc: 82.64840182648402
Epoch: [2/10], Step: [201/313], Validation Acc: 84.17047184170472
Epoch: [2/10], Step: [301/313], Validation Acc: 86.5296803652968
Epoch: [3/10], Step: [101/313], Validation Acc: 86.91019786910198
Epoch: [3/10], Step: [201/313], Validation Acc: 88.43226788432268
Epoch: [3/10], Step: [301/313], Validation Acc: 87.8234398782344
Epoch: [4/10], Step: [101/313], Validation Acc: 88.0517503805175
Epoch: [4/10], Step: [201/313], Validation Acc: 88.81278538812785
Epoch: [4/10], Step: [301/313], Validation Acc: 88.35616438356165
Epoch: [5/10], Step: [101/313], Validation Acc: 88.58447488584476
Epoch: [5/10], Step: [201/313], Validation Acc: 88.58447488584476
Epoch: [5/10], Step: [301/313], Validation Acc: 88.73668188736681
Epoch: [6/10]

In [15]:
print ("After training for {} epochs".format(num_epochs))
print ("Val Acc {}".format(test_model(val_loader, model)))
print ("Test Acc {}".format(test_model(test_loader, model)))

After training for 10 epochs
Val Acc 87.67123287671232
Test Acc 78.86351566648965


## Exercise 1
### Try training the model with larger embedding size and for larger number of epochs
### Also plot the training curves of the model

## Exercise 2:
### Try downloading IMDB Large Movie Review Dataset that is used for Assignment 1 http://ai.stanford.edu/~amaas/data/sentiment/
### and tokenize it

## Exercise 3:
### If you have time, after tokenizing the dataset try training Bag-of-Words model on it and report your initial results
### on validation set.